# Quickstart

Tutorial URL: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Valid as of: 2023.05.04

## Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import ml.core.repo_paths
import os

# Download datasets

In [2]:
data_path = ml.core.repo_paths.get_dir_artifacts_data_cache("tutorial_pytorch_quickstart")

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root=data_path,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=data_path,
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:00<00:00, 28267454.65it/s]


Extracting c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw\train-images-idx3-ubyte.gz to c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 662004.72it/s]


Extracting c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw\train-labels-idx1-ubyte.gz to c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 10487584.97it/s]


Extracting c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to c:\Users\sophi\Code\ml\artifacts\data\tutorial_pytorch_quickstart\cache\FashionMNIST\raw



# Creating data loaders

Datasets seem to be definitions. Data loaders seem to be iterators through those datasets.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating models

We start by detecting the device to use for training:

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


Then we actually define the model:

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

And we create an instance of the model on our device:

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Training

We need a loss function and an optimizer:

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

And a training loop:

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

And a testing loop:

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Let's train and test!

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306752  [   64/60000]
loss: 2.295915  [ 6464/60000]
loss: 2.270554  [12864/60000]
loss: 2.275159  [19264/60000]
loss: 2.256564  [25664/60000]
loss: 2.224306  [32064/60000]
loss: 2.238765  [38464/60000]
loss: 2.202085  [44864/60000]
loss: 2.204576  [51264/60000]
loss: 2.177479  [57664/60000]
Test Error: 
 Accuracy: 44.9%, Avg loss: 2.166667 

Epoch 2
-------------------------------
loss: 2.169057  [   64/60000]
loss: 2.171366  [ 6464/60000]
loss: 2.100877  [12864/60000]
loss: 2.140483  [19264/60000]
loss: 2.092038  [25664/60000]
loss: 2.011554  [32064/60000]
loss: 2.066710  [38464/60000]
loss: 1.972531  [44864/60000]
loss: 1.992507  [51264/60000]
loss: 1.942095  [57664/60000]
Test Error: 
 Accuracy: 53.1%, Avg loss: 1.920176 

Epoch 3
-------------------------------
loss: 1.934716  [   64/60000]
loss: 1.925624  [ 6464/60000]
loss: 1.788475  [12864/60000]
loss: 1.868341  [19264/60000]
loss: 1.757584  [25664/60000]
loss: 1.677427  [32064/600

# Saving and loading

We can save our model:

In [14]:
os.makedirs(ml.core.repo_paths.get_dir_models("tutorial_pytorch_quickstart"), exist_ok=True)

model_path = ml.core.repo_paths.get_dir_models("tutorial_pytorch_quickstart") / "model.pth"

torch.save(model.state_dict(), model_path)
           
print(f"Saved PyTorch Model State to {model_path}")

Saved PyTorch Model State to c:\Users\sophi\Code\ml\artifacts\models\tutorial_pytorch_quickstart\model.pth


And we can load it again:

In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Lastly, let's use our loaded model to make predictions:

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
